In [ ]:
import numpy as np
from main_6sept import LeapNode_Poscontrol, LeapNode_Taucontrol
import time
import mujoco
from leap_hand_utils.dynamixel_client import *
import numpy as np
import pandas as pd


In [ ]:
# Initialize dictionaries to store data
df = pd.DataFrame(columns=['Time','start_time', 'Torque', 'Position','Velocity','Control_mode','subtask'])



In [ ]:
#single object
leap_hand = LeapNode_Poscontrol()
pos=leap_hand.read_pos_leap()
print(pos)
pos=np.array([ 0.11664115,  0.91737925,  1.06617533, -0.3604268,   0.210214,    1.63988434,
 -0.44172777,  0.00926267,  0.12584506,  1.62607847, -0.45860161, -0.13032974,
  1.57699095, -0.07970823,  0.85295211, -0.85743655]
)


In [ ]:
#object in palm
leap_hand = LeapNode_Poscontrol()
pos2=leap_hand.read_pos_leap()
print(pos2)


pos2=np.array([-0.07050456,  0.91277729,  0.69495188, -0.02755298,  0.07829176,  1.66596161,
  1.31007849,  0.08596169,  0.05221401,  1.6030687,   1.38677727,  0.01233064,
  1.48955379, -0.18555297,  0.78699075, -0.76079573]
)




In [ ]:
#both object
leap_hand = LeapNode_Poscontrol()
pos3=leap_hand.read_pos_leap()
print(pos3)

pos3=np.array([-0.05823268,  1.49568974,  0.24856363, -0.06436862,  0.02613626,  1.33001982,
  1.45273863,  0.24242769, -0.00761117,  1.24565111,  1.46501051,  0.37895189,
  1.42052494,  0.07522379,  0.11510716, -0.15947522])


In [ ]:
def J(model,data,site_name):
        # model=mujoco.MjModel.from_xml_path(xml_path)
        # data = mujoco.MjData(model)
        mujoco.mj_forward(model, data)
        jacp = np.zeros((3, model.nv))  # translation jacobian
        jacr = np.zeros((3, model.nv)) 

        site_id=model.site(site_name).id
        mujoco.mj_jacSite(model, data, jacp, jacr, site_id)

        return np.vstack((jacp, jacr))

In [ ]:
pos_index= pos[0:4].reshape(4)
pos_index_mujoco=[pos_index[1], pos_index[0], pos_index[2], pos_index[3]]

pos_thumb=pos_thumb_mujoco= pos[12:16].reshape(4)


index_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/index_finger.xml'

index_m = mujoco.MjModel.from_xml_path(index_model_path)
index_d = mujoco.MjData(index_m)


index_d.qpos= pos_index_mujoco
mujoco.mj_forward(index_m, index_d)
index_J=J(index_m,index_d,'contact_index')


thumb_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/thumb.xml'

thumb_m = mujoco.MjModel.from_xml_path(thumb_model_path)
thumb_d = mujoco.MjData(thumb_m)

thumb_d.qpos=pos_thumb_mujoco
mujoco.mj_forward(thumb_m, thumb_d)
thumb_J=J(thumb_m,thumb_d,'contact_thumb')



pos_middle= pos2[4:8].reshape(4)
pos_middle_mujoco=[pos_middle[1], pos_middle[0], pos_middle[2], pos_middle[3]]

middle_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/middle.xml'

middle_m = mujoco.MjModel.from_xml_path(middle_model_path)
middle_d = mujoco.MjData(middle_m)


middle_d.qpos= pos_middle_mujoco
mujoco.mj_forward(middle_m, middle_d)
middle_J=J(middle_m,middle_d,'contact_middle')



pos_tertiary= pos[8:12].reshape(4)
pos_tertiary_mujoco=[pos_tertiary[1], pos_tertiary[0], pos_tertiary[2], pos_tertiary[3]]

tertiary_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/tertiary.xml'

tertiary_m = mujoco.MjModel.from_xml_path(tertiary_model_path)
tertiary_d = mujoco.MjData(tertiary_m)


tertiary_d.qpos= pos_tertiary_mujoco
mujoco.mj_forward(tertiary_m, tertiary_d)
tertiary_J=J(tertiary_m,tertiary_d,'contact_tertiary')


pos_index2= pos3[0:4].reshape(4)
pos_index2_mujoco=[pos_index2[1], pos_index2[0], pos_index2[2], pos_index2[3]]

pos_thumb2=pos_thumb2_mujoco= pos3[-4:].reshape(4)


index_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/index_finger.xml'

index_m = mujoco.MjModel.from_xml_path(index_model_path)
index_d = mujoco.MjData(index_m)


index_d.qpos= pos_index2_mujoco
mujoco.mj_forward(index_m, index_d)
index_J2=J(index_m,index_d,'contact_index')


thumb_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/thumb.xml'

thumb_m = mujoco.MjModel.from_xml_path(thumb_model_path)
thumb_d = mujoco.MjData(thumb_m)

thumb_d.qpos=pos_thumb2_mujoco
mujoco.mj_forward(thumb_m, thumb_d)
thumb_J2=J(thumb_m,thumb_d,'contact_thumb')

# print(index_J2)
# print(thumb_J2)

In [ ]:
F_index1 = np.reshape([-0.20, 0, 0, 0, 0, 0], [6, 1])
F_thumb1=np.reshape([0.20, 0, 0, 0, 0, 0], [6, 1])

# Compute torque values
Tau_index1 = index_J.T @ F_index1
Tau_index1[[0, 1]] = Tau_index1[[1, 0]]

Tau_thumb1 = thumb_J.T @ F_thumb1

# Convert torque values to float
Tau_index1 = [float(torque[0]) for torque in Tau_index1]
Tau_thumb1 = [float(torque[0]) for torque in Tau_thumb1]


F_index2 = np.reshape([-0.12, 0, 0, 0, 0, 0], [6, 1])
F_thumb2=np.reshape([0.12, 0, 0, 0, 0, 0], [6, 1])

# Compute torque values
Tau_index2 = index_J.T @ F_index2
Tau_index2[[0, 1]] = Tau_index2[[1, 0]]

Tau_thumb2 = thumb_J.T @ F_thumb2

# Convert torque values to float
Tau_index2 = [float(torque[0]) for torque in Tau_index2]
Tau_thumb2 = [float(torque[0]) for torque in Tau_thumb2]

Start from here

In [1]:
import numpy as np
from main_6sept import LeapNode_Poscontrol, LeapNode_Taucontrol
import time
import mujoco
from leap_hand_utils.dynamixel_client import *
import numpy as np
import pandas as pd

# Initialize dictionaries to store data
df = pd.DataFrame(columns=['Time','start_time', 'Torque', 'Position','Velocity','Control_mode','subtask'])


#single object
leap_hand = LeapNode_Poscontrol()
pos=leap_hand.read_pos_leap()

pos=np.array([ 0.11664115,  0.91737925,  1.06617533, -0.3604268,   0.210214,    1.63988434,
 -0.44172777,  0.00926267,  0.12584506,  1.62607847, -0.45860161, -0.13032974,
  1.57699095, -0.07970823,  0.85295211, -0.85743655]
)

#object in palm
leap_hand = LeapNode_Poscontrol()
pos2=leap_hand.read_pos_leap()


pos2=np.array([-0.07050456,  0.91277729,  0.69495188, -0.02755298,  0.07829176,  1.66596161,
  1.31007849,  0.08596169,  0.05221401,  1.6030687,   1.38677727,  0.01233064,
  1.48955379, -0.18555297,  0.78699075, -0.76079573]
)


def J(model,data,site_name):
	# model=mujoco.MjModel.from_xml_path(xml_path)
	# data = mujoco.MjData(model)
	mujoco.mj_forward(model, data)
	jacp = np.zeros((3, model.nv))  # translation jacobian
	jacr = np.zeros((3, model.nv))

	site_id=model.site(site_name).id
	mujoco.mj_jacSite(model, data, jacp, jacr, site_id)

	return np.vstack((jacp, jacr))
pos_index= pos[0:4].reshape(4)
pos_index_mujoco=[pos_index[1], pos_index[0], pos_index[2], pos_index[3]]

pos_thumb=pos_thumb_mujoco= pos[12:16].reshape(4)


index_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/index_finger.xml'

index_m = mujoco.MjModel.from_xml_path(index_model_path)
index_d = mujoco.MjData(index_m)


index_d.qpos= pos_index_mujoco
mujoco.mj_forward(index_m, index_d)
index_J=J(index_m,index_d,'contact_index')


thumb_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/thumb.xml'

thumb_m = mujoco.MjModel.from_xml_path(thumb_model_path)
thumb_d = mujoco.MjData(thumb_m)

thumb_d.qpos=pos_thumb_mujoco
mujoco.mj_forward(thumb_m, thumb_d)
thumb_J=J(thumb_m,thumb_d,'contact_thumb')



pos_middle= pos2[4:8].reshape(4)
pos_middle_mujoco=[pos_middle[1], pos_middle[0], pos_middle[2], pos_middle[3]]

middle_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/middle.xml'

middle_m = mujoco.MjModel.from_xml_path(middle_model_path)
middle_d = mujoco.MjData(middle_m)


middle_d.qpos= pos_middle_mujoco
mujoco.mj_forward(middle_m, middle_d)
middle_J=J(middle_m,middle_d,'contact_middle')



pos_tertiary= pos[8:12].reshape(4)
pos_tertiary_mujoco=[pos_tertiary[1], pos_tertiary[0], pos_tertiary[2], pos_tertiary[3]]

tertiary_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/tertiary.xml'

tertiary_m = mujoco.MjModel.from_xml_path(tertiary_model_path)
tertiary_d = mujoco.MjData(tertiary_m)


tertiary_d.qpos= pos_tertiary_mujoco
mujoco.mj_forward(tertiary_m, tertiary_d)
tertiary_J=J(tertiary_m,tertiary_d,'contact_tertiary')


pos_index2= pos2[0:4].reshape(4)
pos_index2_mujoco=[pos_index2[1], pos_index2[0], pos_index2[2], pos_index2[3]]

pos_thumb2=pos_thumb2_mujoco= pos2[-4:].reshape(4)


index_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/index_finger.xml'

index_m = mujoco.MjModel.from_xml_path(index_model_path)
index_d = mujoco.MjData(index_m)


index_d.qpos= pos_index2_mujoco
mujoco.mj_forward(index_m, index_d)
index_J2=J(index_m,index_d,'contact_index')


thumb_model_path = '/home/saniya/LEAP/leap_hand_mujoco/model/leap hand/thumb.xml'

thumb_m = mujoco.MjModel.from_xml_path(thumb_model_path)
thumb_d = mujoco.MjData(thumb_m)

thumb_d.qpos=pos_thumb2_mujoco
mujoco.mj_forward(thumb_m, thumb_d)
thumb_J2=J(thumb_m,thumb_d,'contact_thumb')

# print(index_J2)
# print(thumb_J2)
F_index1 = np.reshape([-0.15, 0, 0, 0, 0, 0], [6, 1])
F_thumb1=np.reshape([0.15, 0, 0, 0, 0, 0], [6, 1])

# Compute torque values
Tau_index1 = index_J.T @ F_index1
Tau_index1[[0, 1]] = Tau_index1[[1, 0]]

Tau_thumb1 = thumb_J.T @ F_thumb1

# Convert torque values to float
Tau_index1 = [float(torque[0]) for torque in Tau_index1]
Tau_thumb1 = [float(torque[0]) for torque in Tau_thumb1]


F_index2 = np.reshape([-0.1, 0, 0, 0, 0, 0], [6, 1])
F_thumb2=np.reshape([0.1, 0, 0, 0, 0, 0], [6, 1])

# Compute torque values
Tau_index2 = index_J.T @ F_index2
Tau_index2[[0, 1]] = Tau_index2[[1, 0]]

Tau_thumb2 = thumb_J.T @ F_thumb2

# Convert torque values to float
Tau_index2 = [float(torque[0]) for torque in Tau_index2]
Tau_thumb2 = [float(torque[0]) for torque in Tau_thumb2]


[DEBUG] [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'
[DEBUG] [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


In [12]:
leap_pos = LeapNode_Poscontrol()

q0 = np.zeros(16) # 16-element array
v0 = np.zeros(16)  # initial velocity (array of zeros)
q1 = pos.reshape(16)  # final positions
v1 = np.zeros(16)  # final velocity (array of zeros)
t0 = 3 # initial time
t1 = 7

# q2=np.array([a1,b1,c1,d1, 0,0,0,0,0,0,0,0,e1,f1,g1,h1])
# v2=np.zeros(16)

start_time = time.time()

while time.time() - start_time < t0:
    leap_pos.set_allegro(np.zeros(16))
    # Create a new row to append
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_pos.read_cur()],
        'Position': [leap_pos.read_pos()],
        'Velocity': [leap_pos.read_vel()],
        'Control_mode': ['pos'],
        'Subtask': ['start']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)
    time.sleep(0.03)

while time.time() - start_time < t1 and time.time() - start_time>t0:
    current_time=time.time()
    # Ensure the current time falls within [t0, t1]
    
    qd = leap_pos.cubic_trajectory(q0, v0, q1, v1, t0, t1, current_time)
    leap_pos.set_allegro(qd)
    # Create a new row to append
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_pos.read_cur()],
        'Position': [leap_pos.read_pos()],
        'Velocity': [leap_pos.read_vel()],
        'Control_mode': ['pos'],
        'Subtask': ['touch_object1']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)
    # time.sleep(0.03)

leap_torque=LeapNode_Taucontrol()


# Apply torque
while time.time() - start_time < 15 and time.time() - start_time > t1:
    curr_pos = leap_torque.read_pos_leap()
    
    # After updating Tau_thumb, store current position for the next time step
    Tau_thumb1[0] = 0.3*((pos[12] - curr_pos[12]))
    leap_torque.set_desired_torque(Tau_index1+ [0,-0.1,0,0,0,-0.1,0,0]+Tau_thumb1)
    # Create a new row to append
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_torque.read_cur()],
        'Position': [leap_torque.read_pos()],
        'Velocity': [leap_torque.read_vel()],
        'Control_mode': ['Tau'],
        'Subtask': ['grasp_object1']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)



[DEBUG] [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!


[DEBUG] [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!


In [13]:
start_time=time.time()
while time.time() - start_time < 2 and time.time() - start_time > 0:
    
    leap_torque.set_desired_torque(Tau_index2+ [0,-0.1,0,0,0,-0.1,0,0]+Tau_thumb2)
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_torque.read_cur()],
        'Position': [leap_torque.read_pos()],
        'Velocity': [leap_torque.read_vel()],
        'Control_mode': ['Tau'],
        'Subtask': ['readyfordrop']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)


ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!


In [14]:
# # Scaling factor for gradual reduction
# scale_factor = 0.99  # Reduce by 1% in each loop iteration

scale_factor = 0.999  # Reduce by 1% in each loop iteration
F_index=F_index2
F_thumb=F_thumb2

start_time=time.time()

while 0 <= (time.time() - start_time) <= 2:
    # Gradually reduce the forces
    F_index = scale_factor * F_index
    F_thumb = scale_factor * F_thumb

    # Recompute the torque values
    Tau_index = index_J.T @ F_index
    Tau_index[[0, 1]] = Tau_index[[1, 0]]

    Tau_thumb = thumb_J.T @ F_thumb

    # Convert torque values to float
    Tau_index = [float(torque[0]) for torque in Tau_index]
    Tau_thumb = [float(torque[0]) for torque in Tau_thumb]

    # Apply the updated torque
    leap_torque.set_desired_torque(Tau_index + [0, -0.1, 0, 0, 0, -0.1, 0, 0] + Tau_thumb)
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_torque.read_cur()],
        'Position': [leap_torque.read_pos()],
        'Velocity': [leap_torque.read_vel()],
        'Control_mode': ['Tau'],
        'Subtask': ['dropping']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)

leap_pos=LeapNode_Poscontrol()

while 2 <= (time.time() - start_time) <= 4:
    
    leap_pos.set_allegro([0,np.pi/2,0,0,0,0,0,0,0,0,0,0,np.pi/2,0,0,0])
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_pos.read_cur()],
        'Position': [leap_pos.read_pos()],
        'Velocity': [leap_pos.read_vel()],
        'Control_mode': ['pos'],
        'Subtask': ['pi/2']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)

# while 32 <= (time.time() - start_time) <= 35:
#     print("retract1")
#     # Gradually reduce the forces
#     F_index = scale_factor * F_index
#     F_thumb = scale_factor * F_thumb

#     # Recompute the torque values
#     Tau_index = index_J.T @ F_index
#     Tau_index[[0, 1]] = Tau_index[[1, 0]]

#     Tau_thumb = thumb_J.T @ F_thumb

#     # Convert torque values to float
#     Tau_index = [float(torque[0]) for torque in Tau_index]
#     Tau_thumb = [float(torque[0]) for torque in Tau_thumb]

#     # Apply the updated torque
#     leap_torque.set_desired_torque(Tau_index + [0, -0.1, 0, 0, 0, -0.1, 0, 0] + Tau_thumb)

# leap_pos=LeapNode_Poscontrol()



[DEBUG] [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


ERROR:root:> read: [TxRxResult] There is no status packet!


In [15]:
leap_pos=LeapNode_Poscontrol()
start_time = time.time()

while time.time() - start_time < 5 and time.time() - start_time>0:
    
    # Ensure the current time falls within [t0, t1]
    
    leap_pos.set_allegro(pos2)
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_pos.read_cur()],
        'Position': [leap_pos.read_pos()],
        'Velocity': [leap_pos.read_vel()],
        'Control_mode': ['pos'],
        'Subtask': ['touch_in_palm']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)
    

leap_torque=LeapNode_Taucontrol()
start_time = time.time()

# middle_J=np.array([[-7.81501428e-02, -1.55539256e-04, -2.51796518e-02,  7.54582854e-03],
#  [ 0.00000000e+00, -3.54369962e-02,  2.48025991e-03,  1.83490060e-03],
#  [ 6.27285238e-02,  2.20242886e-03,  6.56215493e-02,  5.03953729e-02],
#  [ 0.00000000e+00, -9.97515578e-01,  2.59705727e-03,  2.59705727e-03],
#  [ 1.00000000e+00,  0.00000000e+00,  9.99320225e-01,  9.99320225e-01],
#  [ 0.00000000e+00, -7.04462393e-02, -3.67742140e-02, -3.67742140e-02]])

# tertiary_J= np.array([[-8.88466905e-02, -1.70852794e-04, -3.60211914e-02, -6.41372384e-03],
#  [ 0.00000000e+00, -4.24803993e-02,  1.05205027e-03 , 7.20497073e-04],
#  [ 6.69320190e-02,  1.47925465e-03,  7.22206642e-02,  5.15688378e-02],
#  [ 0.00000000e+00, -9.93395963e-01,  1.59069924e-03,  1.59069924e-03],
#  [ 1.00000000e+00,  0.00000000e+00,  9.99903891e-01,  9.99903891e-01],
#  [ 0.00000000e+00, -1.14736483e-01, -1.37723779e-02, -1.37723779e-02]])

F_middle = np.reshape([0, 0, 0.25, 0, 0, 0], [6, 1])
F_tertiary = np.reshape([0, 0, -0.25, 0, 0, 0], [6, 1])

# Compute torque values
Tau_middle = middle_J.T @ F_middle
Tau_middle[[0, 1]] = Tau_middle[[1, 0]]

# Convert torque values to float
Tau_middle = [float(torque[0]) for torque in Tau_middle]

Tau_tertiary = tertiary_J.T @ F_tertiary
Tau_tertiary[[0, 1]] = Tau_tertiary[[1, 0]]

# Convert torque values to float
Tau_tertiary = [float(torque[0]) for torque in Tau_tertiary]

    
while time.time() - start_time < 10 and time.time()>0:
    leap_torque.set_desired_torque([0,0,0,0] + Tau_middle + Tau_tertiary+ [0,0,0,0])
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_torque.read_cur()],
        'Position': [leap_torque.read_pos()],
        'Velocity': [leap_torque.read_vel()],
        'Control_mode': ['Tau'],
        'Subtask': ['grasp_in_palm']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)

[DEBUG] [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!


[DEBUG] [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!


In [16]:
# leap_pos=LeapNode_Poscontrol()
# start_time = time.time()

# while time.time() - start_time < 5 and time.time() - start_time>0:
    
#     # Ensure the current time falls within [t0, t1]
    
#     leap_pos.set_allegro(pos3)
#     new_row = pd.DataFrame({
#         'Time': [time.time()],
#         'start_time': [start_time],
#         'Torque': [leap_pos.read_cur()],
#         'Position': [leap_pos.read_pos()],
#         'Velocity': [leap_pos.read_vel()],
#         'Control_mode': ['pos'],
#         'Subtask': ['second_object_touch']
#     })

#     # Append the new row using concat
#     if not new_row.empty:
#         df = pd.concat([df, new_row], ignore_index=True)
    
start_time = time.time()

# middle_J=np.array([[-7.81501428e-02, -1.55539256e-04, -2.51796518e-02,  7.54582854e-03],
#  [ 0.00000000e+00, -3.54369962e-02,  2.48025991e-03,  1.83490060e-03],
#  [ 6.27285238e-02,  2.20242886e-03,  6.56215493e-02,  5.03953729e-02],
#  [ 0.00000000e+00, -9.97515578e-01,  2.59705727e-03,  2.59705727e-03],
#  [ 1.00000000e+00,  0.00000000e+00,  9.99320225e-01,  9.99320225e-01],
#  [ 0.00000000e+00, -7.04462393e-02, -3.67742140e-02, -3.67742140e-02]])

# tertiary_J= np.array([[-8.88466905e-02, -1.70852794e-04, -3.60211914e-02, -6.41372384e-03],
#  [ 0.00000000e+00, -4.24803993e-02,  1.05205027e-03 , 7.20497073e-04],
#  [ 6.69320190e-02,  1.47925465e-03,  7.22206642e-02,  5.15688378e-02],
#  [ 0.00000000e+00, -9.93395963e-01,  1.59069924e-03,  1.59069924e-03],
#  [ 1.00000000e+00,  0.00000000e+00,  9.99903891e-01,  9.99903891e-01],
#  [ 0.00000000e+00, -1.14736483e-01, -1.37723779e-02, -1.37723779e-02]])

F_index_final = np.reshape([-0.15, 0, 0, 0, 0, 0], [6, 1])
F_thumb_final = np.reshape([0.15, 0, 0, 0, 0, 0], [6, 1])

# Compute torque values
Tau_index_final = index_J2.T @ F_index_final
Tau_index_final[[0, 1]] = Tau_index_final[[1, 0]]

# Convert torque values to float
Tau_index_final = [float(torque[0]) for torque in Tau_index_final]

Tau_thumb_final = thumb_J2.T @ F_thumb_final

# Convert torque values to float
Tau_thumb_final = [float(torque[0]) for torque in Tau_thumb_final]

    
while time.time() - start_time < 2 and time.time()>0:
    leap_torque.set_desired_torque([0,0,0,0] + Tau_middle + Tau_tertiary+ [0,0,0,0])
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_torque.read_cur()],
        'Position': [leap_torque.read_pos()],
        'Velocity': [leap_torque.read_vel()],
        'Control_mode': ['Tau'],
        'Subtask': ['second_object_grasp']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)

while time.time() - start_time < 10 and time.time()>2:
    leap_torque.set_desired_torque(Tau_index_final + Tau_middle + Tau_tertiary+ Tau_thumb_final)
    new_row = pd.DataFrame({
        'Time': [time.time()],
        'start_time': [start_time],
        'Torque': [leap_torque.read_cur()],
        'Position': [leap_torque.read_pos()],
        'Velocity': [leap_torque.read_vel()],
        'Control_mode': ['Tau'],
        'Subtask': ['second_object_grasp']
    })

    # Append the new row using concat
    if not new_row.empty:
        df = pd.concat([df, new_row], ignore_index=True)

ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!
ERROR:root:> read: [TxRxResult] There is no status packet!


In [17]:
# After the loop ends, save DataFrame to CSV
csv_file_path = '/home/saniya/LEAP/multi-object-grasp/python/stored_data/3_4.csv'  # Specify the correct path
df.to_csv(csv_file_path, index=False)